## Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
# Check the shape of Dataframe
print(application_df.shape)

(34299, 12)


### Identification of Variables
y = application_df["IS_SUCCESSFUL"].values
X = application_df.drop(columns = ["IS_SUCCESSFUL"], axis=1)

In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(columns = ["EIN","NAME"], axis=1)
application_df

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1
...,...,...,...,...,...,...,...,...,...,...
34294,T4,Independent,C1000,ProductDev,Association,1,0,N,5000,0
34295,T4,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
34296,T3,CompanySponsored,C2000,Preservation,Association,1,0,N,5000,0
34297,T5,Independent,C3000,ProductDev,Association,1,0,N,5000,1


In [3]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [6]:
# Look at APPLICATION_TYPE value counts for binning
application_df["APPLICATION_TYPE"].value_counts()

APPLICATION_TYPE
T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: count, dtype: int64

In [4]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
# Choose a cutoff value
cutoff_value = 500

# Create a list of application types to be replaced
application_types_to_replace = application_df['APPLICATION_TYPE'].value_counts()[application_df['APPLICATION_TYPE'].value_counts() < cutoff_value].index.tolist()


# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: count, dtype: int64

In [5]:
# Look at CLASSIFICATION value counts for binning
application_df["CLASSIFICATION"].value_counts()

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: count, Length: 71, dtype: int64

In [6]:
# You may find it helpful to look at CLASSIFICATION value counts >1
Classification_count = application_df["CLASSIFICATION"].value_counts()

classification_bin = Classification_count.loc[Classification_count>1]
classification_bin


CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: count, dtype: int64

In [7]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
# Choose a cutoff value
cutoff_value = 1800

# Create a list of application types to be replaced
classifications_to_replace = application_df["CLASSIFICATION"].value_counts()[application_df["CLASSIFICATION"].value_counts() < cutoff_value].index.tolist()

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: count, dtype: int64

In [8]:
# Look at APPLICATION_TYPE value counts for binning
application_df["INCOME_AMT"].value_counts()

INCOME_AMT
0                24388
25000-99999       3747
100000-499999     3374
1M-5M              955
1-9999             728
10000-24999        543
10M-50M            240
5M-10M             185
50M+               139
Name: count, dtype: int64

In [9]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
# Choose a cutoff value
cutoff_value = 500

# Create a list of application types to be replaced
amount_to_replace = application_df["INCOME_AMT"].value_counts()[application_df["INCOME_AMT"].value_counts() < cutoff_value].index.tolist()

# Replace in dataframe
for amt in amount_to_replace:
    application_df['INCOME_AMT'] = application_df['INCOME_AMT'].replace(amt,"Other")
    
# Check to make sure binning was successful
application_df['INCOME_AMT'].value_counts()

INCOME_AMT
0                24388
25000-99999       3747
100000-499999     3374
1M-5M              955
1-9999             728
Other              564
10000-24999        543
Name: count, dtype: int64

In [10]:
# Convert categorical data to numeric with `pd.get_dummies`
application_df = pd.get_dummies(application_df).astype(int)
application_df

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,ORGANIZATION_Trust,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_Other,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0,1,0,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0
1,1,108590,1,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,1,0
2,1,5000,0,0,0,0,0,0,1,0,...,0,1,0,0,0,0,0,0,1,0
3,1,6692,1,0,0,0,1,0,0,0,...,1,0,0,1,0,0,0,0,1,0
4,1,142590,1,0,0,0,1,0,0,0,...,1,0,0,0,1,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,1,5000,0,0,0,0,0,1,0,0,...,0,1,0,0,0,0,0,0,1,0
34295,1,5000,0,0,0,0,0,1,0,0,...,0,1,0,0,0,0,0,0,1,0
34296,1,5000,0,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,1,0
34297,1,5000,1,0,0,0,0,0,1,0,...,0,1,0,0,0,0,0,0,1,0


In [11]:
print(application_df.shape)

(34299, 42)


In [12]:
# Split our preprocessed data into our features and target arrays
y = application_df["IS_SUCCESSFUL"].values
X = application_df.drop(columns=["IS_SUCCESSFUL"], axis = 1).values

In [13]:
# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)


In [14]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Model 1: Compile, Train and Evaluate the Model

In [23]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(80, input_dim=X_train.shape[1], activation='sigmoid'))

# Second hidden layer
nn.add(tf.keras.layers.Dense(30, activation='relu'))
# third hidden layer
nn.add(tf.keras.layers.Dense(30, activation='relu'))

# Output layer
nn.add(tf.keras.layers.Dense(1, activation='sigmoid'))

# Check the structure of the model
nn.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 80)                3360      
                                                                 
 dense_7 (Dense)             (None, 30)                2430      
                                                                 
 dense_8 (Dense)             (None, 30)                930       
                                                                 
 dense_9 (Dense)             (None, 1)                 31        
                                                                 
Total params: 6751 (26.37 KB)
Trainable params: 6751 (26.37 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [24]:
# Compile the model
nn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [25]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
804/804 [==============================] - 12s 10ms/step - loss: 0.5796 - accuracy: 0.7181
Epoch 2/100
804/804 [==============================] - 10s 12ms/step - loss: 0.5592 - accuracy: 0.7296
Epoch 3/100
804/804 [==============================] - 9s 11ms/step - loss: 0.5558 - accuracy: 0.7297
Epoch 4/100
804/804 [==============================] - 7s 9ms/step - loss: 0.5536 - accuracy: 0.7309
Epoch 5/100
804/804 [==============================] - 7s 9ms/step - loss: 0.5528 - accuracy: 0.7332
Epoch 6/100
804/804 [==============================] - 7s 9ms/step - loss: 0.5521 - accuracy: 0.7317
Epoch 7/100
804/804 [==============================] - 8s 10ms/step - loss: 0.5515 - accuracy: 0.7328
Epoch 8/100
804/804 [==============================] - 8s 10ms/step - loss: 0.5498 - accuracy: 0.7330
Epoch 9/100
804/804 [==============================] - 8s 10ms/step - loss: 0.5494 - accuracy: 0.7346
Epoch 10/100
804/804 [==============================] - 8s 9ms/step - loss: 0.5494 

In [26]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 2s - loss: 0.5636 - accuracy: 0.7259 - 2s/epoch - 6ms/step
Loss: 0.563557505607605, Accuracy: 0.7259474992752075


In [19]:
# Export our model to HDF5 file
nn.save('AlphabetSoupCharityM1.h5')

c:\Users\muhka\anaconda3\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


## Model 2: Train Evaluate and Summarize

In [30]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

nn2 = tf.keras.models.Sequential()

# First hidden layer
nn2.add(tf.keras.layers.Dense(90, input_dim=X_train.shape[1], activation='relu'))

# Second hidden layer
nn2.add(tf.keras.layers.Dense(40, activation='relu'))
# third hidden layer
nn2.add(tf.keras.layers.Dense(40, activation='relu'))

# Output layer
nn2.add(tf.keras.layers.Dense(1, activation='sigmoid'))

# Check the structure of the model
nn2.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_14 (Dense)            (None, 90)                3780      
                                                                 
 dense_15 (Dense)            (None, 40)                3640      
                                                                 
 dense_16 (Dense)            (None, 40)                1640      
                                                                 
 dense_17 (Dense)            (None, 1)                 41        
                                                                 
Total params: 9101 (35.55 KB)
Trainable params: 9101 (35.55 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [31]:
# Compile the model
nn2.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [32]:
# Train the model
fit_model2 = nn2.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
804/804 [==============================] - 11s 9ms/step - loss: 0.5685 - accuracy: 0.7253
Epoch 2/100
804/804 [==============================] - 8s 10ms/step - loss: 0.5537 - accuracy: 0.7304
Epoch 3/100
804/804 [==============================] - 9s 11ms/step - loss: 0.5508 - accuracy: 0.7338
Epoch 4/100
804/804 [==============================] - 8s 10ms/step - loss: 0.5493 - accuracy: 0.7336
Epoch 5/100
804/804 [==============================] - 8s 10ms/step - loss: 0.5483 - accuracy: 0.7330
Epoch 6/100
804/804 [==============================] - 8s 11ms/step - loss: 0.5474 - accuracy: 0.7341
Epoch 7/100
804/804 [==============================] - 8s 10ms/step - loss: 0.5463 - accuracy: 0.7355
Epoch 8/100
804/804 [==============================] - 8s 10ms/step - loss: 0.5456 - accuracy: 0.7346
Epoch 9/100
804/804 [==============================] - 8s 10ms/step - loss: 0.5450 - accuracy: 0.7358
Epoch 10/100
804/804 [==============================] - 9s 11ms/step - loss: 0.546

In [33]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn2.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 2s - loss: 0.5823 - accuracy: 0.7256 - 2s/epoch - 6ms/step
Loss: 0.5823444724082947, Accuracy: 0.7255976796150208


## Model 3: Train Evaluate and Summarize

In [34]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

nn3 = tf.keras.models.Sequential()

# First hidden layer
nn3.add(tf.keras.layers.Dense(80, input_dim=X_train.shape[1], activation='sigmoid'))

# Second hidden layer
nn3.add(tf.keras.layers.Dense(30, activation='relu'))
# third hidden layer
nn3.add(tf.keras.layers.Dense(30, activation='relu'))
# fourth hidden layer
nn3.add(tf.keras.layers.Dense(30, activation='relu'))
# Output layer
nn3.add(tf.keras.layers.Dense(1, activation='sigmoid'))

# Check the structure of the model
nn3.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_18 (Dense)            (None, 80)                3360      
                                                                 
 dense_19 (Dense)            (None, 30)                2430      
                                                                 
 dense_20 (Dense)            (None, 30)                930       
                                                                 
 dense_21 (Dense)            (None, 30)                930       
                                                                 
 dense_22 (Dense)            (None, 1)                 31        
                                                                 
Total params: 7681 (30.00 KB)
Trainable params: 7681 (30.00 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [35]:
# Compile the model
nn3.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [36]:
# Train the model
fit_model2 = nn3.fit(X_train_scaled, y_train, epochs=200)

Epoch 1/200
804/804 [==============================] - 12s 10ms/step - loss: 0.5784 - accuracy: 0.7155
Epoch 2/200
804/804 [==============================] - 4s 5ms/step - loss: 0.5598 - accuracy: 0.7288
Epoch 3/200
804/804 [==============================] - 4s 5ms/step - loss: 0.5570 - accuracy: 0.7303
Epoch 4/200
804/804 [==============================] - 5s 6ms/step - loss: 0.5551 - accuracy: 0.7314
Epoch 5/200
804/804 [==============================] - 4s 5ms/step - loss: 0.5540 - accuracy: 0.7327
Epoch 6/200
804/804 [==============================] - 6s 7ms/step - loss: 0.5531 - accuracy: 0.7312
Epoch 7/200
804/804 [==============================] - 9s 11ms/step - loss: 0.5516 - accuracy: 0.7327
Epoch 8/200
804/804 [==============================] - 6s 7ms/step - loss: 0.5512 - accuracy: 0.7335
Epoch 9/200
804/804 [==============================] - 6s 7ms/step - loss: 0.5501 - accuracy: 0.7324
Epoch 10/200
804/804 [==============================] - 8s 10ms/step - loss: 0.5497 - ac

In [38]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn3.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 6s - loss: 0.5765 - accuracy: 0.7244 - 6s/epoch - 22ms/step
Loss: 0.5764651298522949, Accuracy: 0.7244315147399902
